In [37]:
from random import randint
import time
from time import gmtime, mktime
from datetime import datetime
import yfinance as yf
import requests
from bs4 import BeautifulSoup

from investpy.stocks import (
    get_stock_recent_data, get_stock_countries, get_stock_company_profile, get_stock_historical_data
)
from investpy.utils.extra import random_user_agent
import json
import pandas as pd
from urllib import parse
import csv

In [173]:
s = time.strptime('2017-07-01', '%Y-%m-%d')
e = time.strptime('2020-03-15', '%Y-%m-%d')
start_1 = int(mktime(s)) + 32400
end_1 = int(mktime(e)) + 32400

s = time.strptime('2020-01-03', '%Y-%m-%d')
e = time.strptime('2022-06-11', '%Y-%m-%d')
start_2 = int(mktime(s)) + 32400
end_2 = int(mktime(e)) + 32400

print(start_1)
print(end_1)
print(start_2)
print(end_2)

1498867200
1584230400
1578009600
1654905600


In [25]:
stocks_path = "/Users/parkdoyeong/miniforge3/envs/crawling/lib/python3.8/site-packages/investpy/resources/stocks.csv"

In [26]:
df = pd.read_csv(stocks_path)
df = df[df["country"] == "united states"]

In [65]:
def get_hour(tag):
    name = df[df["symbol"] == tag]["tag"].values[0]
    
    req = requests.get(
        f"https://www.investing.com/equities/{name}-chart",
        headers = {
            "User-Agent": random_user_agent(),
            "X-Requested-With": "XMLHttpRequest",
            "Accept": "text/html",
            "Accept-Encoding": "gzip, deflate",
            "Connection": "keep-alive",
        }
    )
    
    soup = BeautifulSoup(req.text, 'html.parser')
    url = soup.select_one("#leftColumn > div.tvChartContainer > div:nth-child(1) > iframe").attrs["src"]
    # print(url)
    url_parsing = parse.urlparse(url)
    
    qs = parse.parse_qs(url_parsing.query)
    carrier = qs['carrier'][0]
    symbol = qs['pair_ID'][0]
    t = qs['time'][0]
    # t = "1655695721"
    
    url1 = f"https://tvc4.investing.com/{carrier}/{t}/1/1/8/history?symbol={symbol}&resolution=60&from={start_1}&to={end_1}"
    url2 = f"https://tvc4.investing.com/{carrier}/{t}/1/1/8/history?symbol={symbol}&resolution=60&from={start_2}&to={end_2}"

    req1 = requests.get(
        url1,
        headers = {
            "User-Agent": random_user_agent(),
            "X-Requested-With": "XMLHttpRequest",
            "Accept": "text/html",
            "Accept-Encoding": "gzip, deflate",
            "Connection": "keep-alive",
        }
    )

    req2 = requests.get(
        url2,
        headers = {
            "User-Agent": random_user_agent(),
            "X-Requested-With": "XMLHttpRequest",
            "Accept": "text/html",
            "Accept-Encoding": "gzip, deflate",
            "Connection": "keep-alive",
        }
    )
    
    price_1 = json.loads(req1.text)
    price_2 = json.loads(req2.text)

    for i in range(len(price_1['t'])):
        price_1['t'][i] = datetime.fromtimestamp(price_1['t'][i] - 46800)

    for i in range(len(price_2['t'])):
        price_2['t'][i] = datetime.fromtimestamp(price_2['t'][i] - 46800)

    price_hour = price_1['t']
    
    Df_price = pd.DataFrame({
        'Time' : price_1['t'] + price_2['t'],
        'Opened' : price_1['o'] + price_2['o'],
        'Closed' : price_1['c'] + price_2['c'],
        'High' : price_1['h'] + price_2['h'],
        'Low' : price_1['l'] + price_2['l'],
        # 'Volume' : price_1['vo'] + price_2['vo']
    })
    
    Df_price.to_csv(f'./hour_data/{tag}.csv')

In [58]:
with open("500_Crawl/corporate.txt", "r") as f:
    lines = f.readlines()

In [62]:
from tqdm import tqdm

In [67]:
for tag in tqdm(lines):
    tag = tag.strip()
    try:
        get_hour(tag)
    except:
        print(tag)

  0%|▌                                                                                                                                | 2/448 [00:07<27:00,  3.63s/it]

BALL


  1%|█▏                                                                                                                               | 4/448 [00:11<18:47,  2.54s/it]

BBWI


  2%|██▎                                                                                                                              | 8/448 [00:22<20:59,  2.86s/it]

...

 95%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████      | 427/448 [26:17<01:10,  3.37s/it]

WBD


 98%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍  | 439/448 [27:13<00:42,  4.75s/it]

WTW


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 448/448 [27:46<00:00,  3.72s/it]


In [68]:
li = ['BALL', 'BBWI', 'BRKB', 'BFB', 'CARR', 'CDAY', 'CEG', 'CTRA', 'HWM', 'J', 'LIN', 'LUMN', 'META','MRNA', 'OGN', 'OTIS', 'PARA', 'PAYC', 'RTX', 'TT', 'TFC', 'VTRS', 'VICI', 'WBD', 'WTW']

In [148]:
co_dict['CEG'] = 'equities/constellation-energy'
co_dict['OTIS'] = 'equities/otis-worldwide-corp'
co_dict['WTW'] = 'equities/willis-group-holdings-plc'

In [141]:
corporates = []
for co in li:
    corporates.append(getInvesting(co))

In [143]:
co_dict = dict(zip(li, corporates))

In [152]:
co_dict['BALL'].split('/')[1]

'ball-corp'

In [172]:
co_dict

{'BALL': 'equities/ball-corp',
 'BBWI': 'equities/limited-brands',
 'BRKB': 'equities/berkshire-hathaway',
 'BFB': 'equities/brown-forman-b',
 'CARR': 'equities/carrier-global-corp',
 'CDAY': 'equities/ceridian-hcm-holding-inc',
 'CEG': 'equities/constellation-energy',
 'CTRA': 'equities/ciputra-develo',
 'HWM': 'equities/howmet-aerospace-inc',
 'J': 'equities/jacobs-engineer',
 'LIN': 'equities/lindian-resources-ltd',
 'LUMN': 'equities/centurylink',
 'META': 'equities/facebook-inc',
 'MRNA': 'equities/moderna',
 'OGN': 'equities/organon-co',
 'OTIS': 'equities/otis-worldwide-corp',
 'PARA': 'equities/viacom-cl-b',
 'PAYC': 'equities/paycom-soft',
 'RTX': 'equities/united-tech',
 'TT': 'equities/t-t-ltd',
 'TFC': 'equities/bb-t-corp',
 'VTRS': 'equities/mylan-inc',
 'VICI': 'equities/vici-properties',
 'WBD': 'equities/discovery-holding-co',
 'WTW': 'equities/willis-group-holdings-plc'}

In [103]:
def getInvesting(ticker):
    target_address = 'https://www.google.com/search?q='+ticker+'+investing'
    html = requests.get(target_address)
    soup = BeautifulSoup(html.text, 'html.parser')
    posts = soup.select('.kCrYT a ')
    #for chooes first herf 
    href = str(posts[0].attrs['href'])
    #then href is 'https:/www.investing/equities/stock&~~~~~~ 
    #so 1.find  start-index : index('equities') , end-index : index('&')
    # and slicing href[start : end]
    start = href.find('equities')
    end = href.find('&')
    # print(href)
    # print(href[start:end])
    return href[start:end]

In [134]:
for l in range(25):
    c = corporate[l].split('/')[1]
    print(c)

ball-corp
limited-brands
berkshire-hathaway
brown-forman-b
carrier-global-corp
ceridian-hcm-holding-inc
ciputra-develo
howmet-aerospace-inc
jacobs-engineer
lindian-resources-ltd
centurylink
facebook-inc
moderna
organon-co
viacom-cl-b
paycom-soft
united-tech
trane-technologies-plc
bb-t-corp
mylan-inc
vici-properties
discovery-holding-co
constellation-energy
otis-worldwide-corp
willis-group-holdings-plc


https://tvc4.investing.com/a45af155c061612034247e27ff2ed977/1655276049/1/1/8/history?symbol=6408&resolution=60&from=1524096000&to=1524182400

https://www.investing.com/equities/apple-computer-inc-chart